In [109]:
import pandas as pd
import time
import json

## 读取文件

In [139]:
with open('all.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    print(json_data.keys())
    print(type(json_data))

dict_keys(['confirmed', 'deaths', 'recovered', 'latest'])
<class 'dict'>


In [144]:
json_data['confirmed'].keys()

dict_keys(['locations', 'latest', 'last_updated', 'source'])

In [207]:
confirmed = json_data['confirmed']['locations']
deaths = json_data['deaths']['locations']
recovered = json_data['recovered']['locations']

In [208]:
print(len(confirmed), len(deaths), len(recovered))

264 264 250


## 处理中国数据（按省）并导出

In [140]:
china_indexes = []
for (index, item) in enumerate(json_data['confirmed']['locations']):
    if item['country'] == 'China':
        china_indexes.append(index)
print(china_indexes)

[49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]


In [142]:
provinces = []
for i in china_indexes[:5]:
    p_dict = {}
    p_dict['country'] = confirmed[i]['country']
    p_dict['country_code'] = confirmed[i]['country_code']
    p_dict['province'] = confirmed[i]['province']
    p_dict['coordinates'] = confirmed[i]['coordinates']
    p_dict['latest'] = {
        'confirmed':confirmed[i]['latest'],
        'deaths':deaths[i]['latest'],
        'recovered':recovered[i]['latest']
    }

    timeline = []
    for date in confirmed[i]['history'].keys():
        date_ = time.strftime("%Y-%m-%d", time.strptime(date, "%m/%d/%y"))
        #     print({'date': date_, 'confirmed': cnt})
        timeline.append({
            'date': date_,
            'confirmed': confirmed[i]['history'][date],
            'deaths': deaths[i]['history'][date],
            'recovered': recovered[i]['history'][date]
        })
    p_dict['timeline'] = timeline

    provinces.append(p_dict)

In [147]:
provinces[0]

{'country': 'China',
 'country_code': 'CN',
 'province': 'Anhui',
 'coordinates': {'lat': '31.8257', 'long': '117.2264'},
 'latest': {'confirmed': 991, 'deaths': 6, 'recovered': 317},
 'timeline': [{'date': '2020-01-22',
   'confirmed': 1,
   'deaths': 0,
   'recovered': 0},
  {'date': '2020-01-23', 'confirmed': 9, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-24', 'confirmed': 15, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-25', 'confirmed': 39, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-26', 'confirmed': 60, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-27', 'confirmed': 70, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-28', 'confirmed': 106, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-29', 'confirmed': 152, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-30', 'confirmed': 200, 'deaths': 0, 'recovered': 0},
  {'date': '2020-01-31', 'confirmed': 237, 'deaths': 0, 'recovered': 0},
  {'date': '2020-02-01', 'confirmed': 297, 'deaths': 0, 'recovered': 0},


In [143]:
with open('china_timeline.json','w',encoding='utf8')as f:
    f.write(str(provinces))

## 处理世界数据（按国家）并导出

In [238]:
DF_confirmed_groups = pd.DataFrame(confirmed).groupby('country')
DF_deaths_groups = pd.DataFrame(deaths).groupby('country')
DF_recovered_groups = pd.DataFrame(recovered).groupby('country')

In [239]:
print(len(DF_confirmed_groups), len(DF_deaths_groups), len(DF_recovered_groups))

185 185 185


In [257]:
cntry_names = list(DF_confirmed_groups.groups.keys())

In [252]:
DF_confirmed_groups.get_group('China')

,country,country_code,province,coordinates,history,latest
49,China,CN,Anhui,"{'lat': '31.8257', 'long': '117.2264'}","{'1/22/20': 1, '1/23/20': 9, '1/24/20': 15, '1...",991
50,China,CN,Beijing,"{'lat': '40.1824', 'long': '116.4142'}","{'1/22/20': 14, '1/23/20': 22, '1/24/20': 36, ...",593
51,China,CN,Chongqing,"{'lat': '30.0572', 'long': '107.874'}","{'1/22/20': 6, '1/23/20': 9, '1/24/20': 27, '1...",579
52,China,CN,Fujian,"{'lat': '26.0789', 'long': '117.9874'}","{'1/22/20': 1, '1/23/20': 5, '1/24/20': 10, '1...",355
53,China,CN,Gansu,"{'lat': '37.8099', 'long': '101.0583'}","{'1/22/20': 0, '1/23/20': 2, '1/24/20': 2, '1/...",139
54,China,CN,Guangdong,"{'lat': '23.3417', 'long': '113.4244'}","{'1/22/20': 26, '1/23/20': 32, '1/24/20': 53, ...",1582
55,China,CN,Guangxi,"{'lat': '23.8298', 'long': '108.7881'}","{'1/22/20': 2, '1/23/20': 5, '1/24/20': 23, '1...",254
56,China,CN,Guizhou,"{'lat': '26.8154', 'long': '106.8748'}","{'1/22/20': 1, '1/23/20': 3, '1/24/20': 3, '1/...",147
57,China,CN,Hainan,"{'lat': '19.1959', 'long': '109.7453'}","{'1/22/20': 4, '1/23/20': 5, '1/24/20': 8, '1/...",168
58,China,CN,Hebei,"{'lat': '39.549', 'long': '116.1306'}","{'1/22/20': 1, '1/23/20': 1, '1/24/20': 2, '1/...",328


In [280]:
countries = []
for cntry in cntry_names:
    df = DF_confirmed_groups.get_group(cntry)
    df_deaths = DF_deaths_groups.get_group(cntry)
    df_recovered = DF_recovered_groups.get_group(cntry)

    dates = list(df.history.iloc[0].keys())
    timeline = []

    for d in dates:
        date_ = time.strftime("%Y-%m-%d", time.strptime(d, "%m/%d/%y"))

        # confirmed each day
        sum_confirmed = 0
        for h in df.history:
            sum_confirmed += h[d]

        # deaths each day
        sum_deaths = 0
        for h in df_deaths.history:
            sum_deaths += h[d]

        # recovered each day
        sum_recovered = 0
        for h in df_recovered.history:
            sum_recovered += h[d]

        timeline.append({
            'date': date_,
            'confirmed': sum_confirmed,
            'deaths': sum_deaths,
            'recovered': sum_recovered
        })
        
    countries.append({
        'country': cntry,
        'country_code': df.iloc[0].country_code,
        'latest': {
            'confirmed': sum_confirmed,
            'deaths': sum_deaths,
            'recovered': sum_recovered
        },
        'timeline': timeline
    })

In [281]:
countries_result = pd.DataFrame(countries)

In [283]:
countries_result.to_json("world_timeline.json", orient='records')